# Improving MICE for Data Imputation: A Methodological and Practical Exploration

## Evaluation
Comparing the performance of the our model to impute missing values.
Our model is compared with the following models:
1. MICE - Multiple Imputation by Chained Equations (The original model we're trying to improve).
2. KNNI - K-Nearest Neighbors Imputation.
3. SICE - Single Imputation with Chained Equations.

For ablation study, we compared several versions of our improvements:
1. Ordered only - MICE where the imputation order is computed using the Bayesian Network structure.
2. correlated variables in regression only - MICE where only the correlated variables are used as features to the linear regression. 

### STEP 0 - Imports and constants

In [1]:
# General
import numpy as np
import pandas as pd
from datetime import datetime
import pickle

# Disable warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Models
from BIMICE import BIMICE
from reparo import \
    MICE, \
    SICE, \
    KNNImputer
from BIMICE import BIMICE

# Metrics
from sklearn.metrics import root_mean_squared_error as RMSE, \
    mean_squared_error as MSE, \
    mean_absolute_error as MAE

metrics_dict = {
    "RMSE": RMSE,
    "MSE": MSE,
    "MAE": MAE
}

models_dict = {
    "OrderBIMICE": BIMICE(order_imputations=True, filter_predicators=False),
    "FilterBIMICE": BIMICE(order_imputations=False, filter_predicators=True),
    "FullBIMICE": BIMICE(order_imputations=True, filter_predicators=True),
    "MICE": MICE(),
    # "SICE": SICE(),
    # "KNN": KNNImputer()
}

baselines_algorithms = ["MICE", "SICE", "KNN"]

### STEP 1 - Load data

#### Define the datasets

In [2]:
DATA_FOLDER = "data/"
FILE_SUFFIX = ".csv"
FRAMINGHAM = {"name": "framingham",
              "numeric_columns": ["age", "education", "cigsPerDay", "BPMeds", "totChol", "sysBP", "diaBP", "heartRate", "glucose"], # TODO - check about education
              }
# TODO - add the additional dataset

#### Choose dataset to work with
Options are FRAMINGHAM or TODO-CONTINUE

In [3]:
dataset = FRAMINGHAM

In [4]:
df = pd.read_csv(DATA_FOLDER + dataset["name"] + FILE_SUFFIX)

In [5]:
print(df.shape)
df.head()

(4240, 16)


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


### STEP 2 - Remove not relevant rows and columns
- Remove rows with missing values.
- Remove columns with non-numeric features.

In [6]:
df = df[dataset["numeric_columns"]]
df.dropna(inplace=True)

# Verify that there are no missing values
print("Total amount of missing values:", df.isnull().sum().sum())
print("New shape:", df.shape)

Total amount of missing values: 0
New shape: (3671, 9)


### STEP 3 - Missing values injection functions
We are using two types of missing values injection functions:
1. inject_missing_completely_at_random - Injection a portion of missing values in random cells in the dataset.
2. inject_missing_per_feature - Injecting a different portion of missing values for each column. 

In [7]:
# Set seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

#### Method 1: Injecting completely at random

In [8]:
def inject_missing_completely_at_random(
        df: pd.DataFrame, 
        missing_rate: float
) -> pd.DataFrame:
    """
    Inject missing values completely at random
    :param df: DataFrame
    :param missing_rate: float
    :return: DataFrame
    """
    df = df.copy()
    mask = np.random.rand(*df.shape) < missing_rate
    df[mask] = np.nan
    return df

#### Method 2: Injecting at random to given columns

In [9]:
def inject_missing_per_feature(df: pd.DataFrame,
                               missing_rate: float,
                               features: list
 ) -> pd.DataFrame:
    """
    Inject missing values per feature
    :param df: DataFrame
    :param missing_rate: float
    :param features: list
    :return: DataFrame
    """
    df = df.copy()
    for feature in features:
        mask = np.random.rand(df.shape[0]) < missing_rate
        df.loc[mask, feature] = np.nan
    return df

In [10]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

def inject_missing_values(
        data: pd.DataFrame,
        columns: list[str],
        rows_severity: float # or list[float]
 ) -> pd.DataFrame:
    """
    Inject missing values into the data.
    
    Parameters
    ----------
    data : DataFrame
        The data to inject missing values into.
        
    columns : list[str]
        The columns to inject missing values into.

    rows_severity : float | list[float]
        The severity of the missing values to inject. If a float, the same severity is applied to all columns. If a list, the severity is applied to each column in order.
        
    Returns
    -------
    DataFrame
        The data with missing values injected.
    """
    assert isinstance(data, pd.DataFrame), 'data must be a pandas DataFrame'
    if isinstance(rows_severity, (int, float)):
        rows_severity = [rows_severity] * len(columns)
    assert isinstance(rows_severity, list) and all(isinstance(severity, (int, float)) and 0 < severity < 1 for severity in rows_severity), 'rows_severity must be between 0 and 1'
    assert isinstance(columns, list) and all(column in data.columns for column in columns), 'not all columns are in the DataFrame'


    result_data = data.copy()

    for current_column, current_rows_severity in zip(columns, rows_severity):
        # Determine the number of missing rows
        missing_rows_count = int(len(data) * current_rows_severity)
        # Avoid cases where none/all rows are cleared
        if missing_rows_count == 0:
            missing_rows_count = 1
        if missing_rows_count == len(data):
            missing_rows_count -= 1
        # Inject missing values into the current column
        current_missing_rows_indices = result_data.sample(missing_rows_count).index
        result_data.loc[current_missing_rows_indices, current_column] = None

    return result_data

### STEP 4 - Evaluations
We evaluate the results using two tests:
1. Over different severities of missing completely at random (ranging from 10% to 40%).
2. Over different number of features with missing values.

#### Test 1 - Different severities of missing completely at random

In [11]:
original_data = df.to_numpy()
# Severity levels
severity_levels = [0.1, 0.2, 0.3, 0.4]

results = {}

for current_severity_level in severity_levels:
    print("Severity level:", current_severity_level)
    current_severity_results = {}
    # Inject missing values
    data_with_missing = inject_missing_completely_at_random(df, current_severity_level).to_numpy()
    missing_features_count = np.sum(np.isnan(data_with_missing))

    for current_model_name, current_model in models_dict.items():
        print("    Model:", current_model_name)
        # Impute missing values
        start_time = datetime.now()
        data_with_imputations = current_model.fit_transform(data_with_missing)
        end_time = datetime.now()
        
        max_predictors_count = len(df.columns) - 1
        predicators_counts = [max_predictors_count] * len(df.columns)
        if current_model_name not in baselines_algorithms:
            data_with_imputations, predicators_counts = data_with_imputations
        
        # Calculate metrics
        current_severity_results[current_model_name] = {metric_name: metric_func(original_data, data_with_imputations) 
                                                        for metric_name, metric_func in metrics_dict.items()}
        # Calculate MAPE
        features_mape_sum = 0
        for column_index, predicators_count in enumerate(predicators_counts):
            features_mape_sum += metrics_dict["MAE"](original_data[:, column_index], data_with_imputations[:, column_index]) / max(max_predictors_count - predicators_count, 1)
        current_severity_results[current_model_name]["time"] = (end_time - start_time).total_seconds() * 1000
        current_severity_results[current_model_name]["MAPE"] = features_mape_sum / missing_features_count
    print(current_severity_results)
    results[current_severity_level] = current_severity_results
    with open(f"{dataset['name']}__MCAR__results.pkl", "wb") as f:
        pickle.dump(results, f)

Severity level: 0.1
    Model: OrdereredOnlyBIMICE
    Model: FilteredOnlyBIMICE
    Model: BIMICE
    Model: MICE
{'OrdereredOnlyBIMICE': {'RMSE': 4.580477920034666, 'MSE': 36.989044023490834, 'MAE': 1.0994796662900086, 'time': 320.53999999999996, 'MAPE': np.float64(0.002976035186950398)}, 'FilteredOnlyBIMICE': {'RMSE': 4.1586412551131415, 'MSE': 31.433144851208358, 'MAE': 0.9920399714791769, 'time': 11.25, 'MAPE': np.float64(0.001037119600354388)}, 'BIMICE': {'RMSE': 4.158743354602798, 'MSE': 31.429843358429952, 'MAE': 0.9921322543261779, 'time': 48.972, 'MAPE': np.float64(0.0010372066327755225)}, 'MICE': {'RMSE': 4.024567309679737, 'MSE': 30.650068946554434, 'MAE': 0.9557552897897613, 'time': 404.235, 'MAPE': np.float64(0.0025870067994309327)}}
Severity level: 0.2
    Model: OrdereredOnlyBIMICE
    Model: FilteredOnlyBIMICE
    Model: BIMICE
    Model: MICE
{'OrdereredOnlyBIMICE': {'RMSE': 6.652312165496635, 'MSE': 74.22089732363679, 'MAE': 2.23414864961531, 'time': 160.111, 'MAPE':

#### Test 2 - Different number of features

In [ ]:
original_data = df.to_numpy()
random_runs = 10
# Severity levels
column_severity_range = (0.05, 0.4)
missing_features_counts = [1, 3, 5, 7, 9]

results = {}

for missing_features_count in missing_features_counts:
    current_results = {}
    print("Amount of columns:", missing_features_count)
    for random_run in range(random_runs):
        print("    Random run:", random_run)
        # Draw columns and severity level
        columns_to_inject_missing = np.random.choice(df.columns, missing_features_count, replace=False)
        severity_level = np.random.uniform(*column_severity_range)
        data_with_missing = inject_missing_per_feature(df, severity_level, columns_to_inject_missing).to_numpy()
        for current_model_name, current_model in models_dict.items():
            print("        Model:", current_model_name)
            # Impute missing values
            start_time = datetime.now()
            data_with_imputations = current_model.fit_transform(data_with_missing)
            end_time = datetime.now()
            
            max_predictors_count = len(df.columns) - 1
            predicators_counts = [max_predictors_count] * len(df.columns)
            if current_model_name not in baselines_algorithms:
                data_with_imputations, predicators_counts = data_with_imputations
            
            # Calculate metrics
            current_results[current_model_name] = {metric_name: 0 for metric_name in (list(metrics_dict.keys()) + ["time", "MAPE"])}
            for metric_name, metric_func in metrics_dict.items():
                current_results[current_model_name][metric_name] += (metric_func(original_data, data_with_imputations) / random_runs) # Average on the fly
            # Calculate MAPE
            features_mape_sum = 0
            for column_index, predicators_count in enumerate(predicators_counts):
                features_mape_sum += metrics_dict["MAE"](original_data[:, column_index], data_with_imputations[:, column_index]) / max(max_predictors_count - predicators_count, 1)
            current_results[current_model_name]["time"] = ((end_time - start_time).total_seconds() * 1000) / random_runs
            current_results[current_model_name]["MAPE"] = (features_mape_sum / missing_features_count) / random_runs
    print(current_results)
    results[missing_features_count] = current_results
    with open(f"{dataset['name']}_per_features__results.pkl", "wb") as f:
        pickle.dump(results, f)

Amount of columns: 1
    Random run: 0
        Model: OrdereredOnlyBIMICE
        Model: FilteredOnlyBIMICE
        Model: BIMICE
        Model: MICE
    Random run: 1
        Model: OrdereredOnlyBIMICE
        Model: FilteredOnlyBIMICE
        Model: BIMICE
        Model: MICE
    Random run: 2
        Model: OrdereredOnlyBIMICE
        Model: FilteredOnlyBIMICE
        Model: BIMICE
        Model: MICE
    Random run: 3
        Model: OrdereredOnlyBIMICE
        Model: FilteredOnlyBIMICE
        Model: BIMICE
        Model: MICE
    Random run: 4
        Model: OrdereredOnlyBIMICE
        Model: FilteredOnlyBIMICE
        Model: BIMICE
        Model: MICE
    Random run: 5
        Model: OrdereredOnlyBIMICE
        Model: FilteredOnlyBIMICE
        Model: BIMICE
        Model: MICE
    Random run: 6
        Model: OrdereredOnlyBIMICE
        Model: FilteredOnlyBIMICE
        Model: BIMICE
        Model: MICE
    Random run: 7
        Model: OrdereredOnlyBIMICE
        Model: Filtered

In [28]:
{count: [f"{model}: {results[count][model]['MAPE']}" for model in results[count]] for count in results}

{1: ['OrdereredOnlyBIMICE: 0.006365666962988424',
  'FilteredOnlyBIMICE: 0.0007957083703735532',
  'BIMICE: 0.0007957083703735532',
  'MICE: 0.0064934490374413175'],
 3: ['OrdereredOnlyBIMICE: 0.14298015037619677',
  'FilteredOnlyBIMICE: 0.04018685774100943',
  'BIMICE: 0.04020417816583048',
  'MICE: 0.10567357996869811'],
 5: ['OrdereredOnlyBIMICE: 0.45399768574802046',
  'FilteredOnlyBIMICE: 0.17755434052029528',
  'BIMICE: 0.17742857111357957',
  'MICE: 0.43290796010811883'],
 7: ['OrdereredOnlyBIMICE: 0.39795829010260947',
  'FilteredOnlyBIMICE: 0.13797769513130756',
  'BIMICE: 0.13801309319812755',
  'MICE: 0.35298774640375546'],
 9: ['OrdereredOnlyBIMICE: 0.4241845381665751',
  'FilteredOnlyBIMICE: 0.15123073894832378',
  'BIMICE: 0.15111013522225225',
  'MICE: 0.38600302143692977']}